In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 기타

In [ ]:
import pandas as pd
import json

# CSV → JSON 변환 경로
csv_path = "/content/drive/MyDrive/project/csvs/train_25.csv"
json_path = "/content/drive/MyDrive/project/csvs/train_25.json"

# CSV 로드
df = pd.read_csv(csv_path)

# 'train' 키로 감싸서 저장
wrapped = {"train": df.to_dict(orient='records')}
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(wrapped, f, ensure_ascii=False, indent=4)

print("✅ A3Net 형식 JSON 저장 완료:", json_path)


In [ ]:
import json

# 경로 설정
train_path = '/content/drive/MyDrive/project/csvs/train_25.json'
val_path = '/content/drive/MyDrive/project/csvs/val.json'
test_path = '/content/drive/MyDrive/project/csvs/test.json'  # 추가된 test 경로
output_path = '/content/drive/MyDrive/project/csvs/annotation_25.json'

# 파일 불러오기
with open(train_path, "r") as f:
    train_data = json.load(f)

with open(val_path, "r") as f:
    val_data = json.load(f)

with open(test_path, "r") as f:
    test_data = json.load(f)

# 병합
combined = {
    "train": train_data["train"] if "train" in train_data else train_data,
    "validate": val_data["validate"] if "validate" in val_data else val_data,
    "test": test_data["test"] if "test" in test_data else test_data
}

# 저장
with open(output_path, "w") as f:
    json.dump(combined, f, indent=2)

print("✅ 병합 완료 →", output_path)


✅ 병합 완료 → /content/drive/MyDrive/project/csvs/annotation_25.json


In [ ]:
import json

with open("/content/drive/MyDrive/project/csvs/annotation.json", "r") as f:
    data = json.load(f)

for split in ["train", "validate", "test"]:
    if split in data:
        for item in data[split]:
            item["id"] = item.pop("image_id")  # 'image_id' -> 'id'

with open("/content/drive/MyDrive/project/csvs/annotation.json", "w") as f:
    json.dump(data, f, indent=2)

print("변환 완료")


변환 완료


In [ ]:
import json

with open("/content/drive/MyDrive/project/csvs/annotation.json", "r") as f:
    data = json.load(f)

# 키 순서 재정렬
for split in ["train", "validate", "test"]:
    if split in data:
        new_list = []
        for item in data[split]:
            new_item = {
                "id": item.pop("image_id") if "image_id" in item else item["id"],
                "report": item["report"]
            }
            new_list.append(new_item)
        data[split] = new_list

with open("/content/drive/MyDrive/project/csvs/annotation.json", "w") as f:
    json.dump(data, f, indent=2)

print("✅ 'id' 먼저 오도록 변환 완료")

✅ 'id' 먼저 오도록 변환 완료


In [ ]:
import json
import os

json_path = "/content/drive/MyDrive/project/csvs/annotation.json"
base_image_dir = "/content/drive/MyDrive/project/images"

# split 별 하위 폴더 매핑
split_to_folder = {
    "train": "train_image",
    "validate": "val_image",     # 실제 폴더명이 validate인지 val인지 확인 필요
    "test": "test_image"
}

with open(json_path, 'r') as f:
    data = json.load(f)

for split in ['train', 'validate', 'test']:
    if split in data:
        image_folder = split_to_folder[split]
        for item in data[split]:
            image_id = item.get('id') or item.get('image_id')
            item['image_path'] = os.path.join(base_image_dir, image_folder, image_id)

with open(json_path, 'w') as f:
    json.dump(data, f, indent=2)

print("✅ split별 image_path 재지정 완료")


✅ split별 image_path 재지정 완료


# 25%

In [ ]:
# ==================== 1. 필요한 라이브러리 설치 ====================
!pip install -q nltk scikit-learn

In [ ]:
# ==================== 2. GitHub 저장소 클론 ====================
!git clone https://github.com/Vinh-AI/A3Net.git
%cd A3Net

Cloning into 'A3Net'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 121 (delta 5), reused 10 (delta 5), pack-reused 109 (from 2)
Receiving objects: 100% (121/121), 70.26 MiB | 15.85 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/A3Net


In [ ]:
# ==================== 3. 경로 설정 (직접 맞게 수정하세요) ====================
image_dir = "/content/drive/MyDrive/project/images/train_image_25"
ann_path = "/content/drive/MyDrive/project/csvs/annotation_25.json"
save_dir = "/content/drive/MyDrive/project/results/25/train_25"

In [ ]:
# ======================== 4. 모델 학습 ========================
!python main.py \
  --image_dir /content/drive/MyDrive/project/images/train_image_25 \
  --ann_path /content/drive/MyDrive/project/csvs/annotation_25.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 64 \
  --epochs 15 \
  --lr_ve 1e-4 \
  --lr_ed 5e-4 \
  --step_size 1 \
  --gamma 0.8 \
  --num_layers 3 \
  --topk 32 \
  --beam_size 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/25/train_25 \
  --log_period 10

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[Image Load] ID: s56678596.jpg - Time: 0.00s
[Image Load] ID: s57606605.jpg - Time: 0.00s
[Image Load] ID: s51643082.jpg - Time: 0.00s
[Image Load] ID: s58803451.jpg - Time: 0.00s
[Image Load] ID: s52501029.jpg - Time: 0.00s
[Image Load] ID: s56834376.jpg - Time: 0.00s
[Image Load] ID: s59611308.jpg - Time: 0.00s
[Image Load] ID: s54814989.jpg - Time: 0.00s
[Image Load] ID: s53506306.jpg - Time: 0.00s
[Image Load] ID: s52668785.jpg - Time: 0.00s
[Image Load] ID: s58101412.jpg - Time: 0.00s
[Image Load] ID: s55129535.jpg - Time: 0.00s
[Image Load] ID: s55280142.jpg - Time: 0.00s
[Image Load] ID: s55864200.jpg - Time: 0.00s
[Image Load] ID: s50678993.jpg - Time: 0.00s
[Image Load] ID: s55316501.jpg - Time: 0.00s
[Image Load] ID: s56515979.jpg - Time: 0.00s
[Image Load] ID: s55278173.jpg - Time: 0.00s
[Image Load] ID: s55772844.jpg - Time: 0.00s
[Image Load] ID: s54082455.jpg - Time: 0.00s
[Image Load] ID: s58956985.jpg - Time: 0.00s
[Image Load] ID: s5

In [8]:
# ======================== 5. 모델 평가 ========================
!python test.py \
  --image_dir /content/drive/MyDrive/project/images/test_image \
  --ann_path /content/drive/MyDrive/project/csvs/annotation_25.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 16 \
  --beam_size 3 \
  --topk 32 \
  --num_layers 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/25/test_25 \
  --load /content/drive/MyDrive/project/results/25/train_25/current_checkpoint.pth

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/08/2025 08:48:56 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/project/results/25/train_25/current_checkpoint.pth ...
06/08/2025 08:49:18 - INFO - modules.tester -   Start to evaluate in the test set.
0it [00:00, ?it/s][Image Load] ID: s50010747.jpg - Time: 0.02s
[Image Load] ID: s50111035.jpg - Time: 0.02s
[Image Load] ID: s50014127.jpg - Time:

# 50%

In [ ]:
# ==================== 1. 필요한 라이브러리 설치 ====================
!pip install -q nltk scikit-learn

In [ ]:
# ==================== 2. GitHub 저장소 클론 ====================
!git clone https://github.com/Vinh-AI/A3Net.git
%cd A3Net

Cloning into 'A3Net'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 121 (delta 5), reused 10 (delta 5), pack-reused 109 (from 2)
Receiving objects: 100% (121/121), 70.26 MiB | 12.36 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/A3Net


In [ ]:
!cp -r /content/drive/MyDrive/project/images/train_image_50 /content/A3Net/data/

cp: cannot access '/content/drive/MyDrive/project/images/train_image_50': Input/output error


In [ ]:
# ==================== 3. 경로 설정 (직접 맞게 수정하세요) ====================
image_dir = "/content/A3Net/data/train_image_50"
ann_path = "/content/drive/MyDrive/project/csvs/annotation_50.json"
save_dir = "/content/drive/MyDrive/project/results/50/train_50"

In [ ]:
# ======================== 4. 모델 학습 ========================
!python main.py \
  --image_dir /content/A3Net/data/train_image_50 \
  --ann_path /content/drive/MyDrive/project/csvs/annotation_50.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 64 \
  --epochs 15 \
  --lr_ve 1e-4 \
  --lr_ed 5e-4 \
  --step_size 1 \
  --gamma 0.8 \
  --num_layers 3 \
  --topk 32 \
  --beam_size 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/50/train_50 \
  --log_period 10

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[Image Load] ID: s51579659.jpg - Time: 0.00s
[Image Load] ID: s54523255.jpg - Time: 0.00s
[Image Load] ID: s57751367.jpg - Time: 0.00s
[Image Load] ID: s52132466.jpg - Time: 0.00s
[Image Load] ID: s59828240.jpg - Time: 0.00s
[Image Load] ID: s57072113.jpg - Time: 0.00s
[Image Load] ID: s59220689.jpg - Time: 0.00s
[Image Load] ID: s55358122.jpg - Time: 0.00s
[Image Load] ID: s56223337.jpg - Time: 0.00s
[Image Load] ID: s50797983.jpg - Time: 0.00s
[Image Load] ID: s54619473.jpg - Time: 0.00s
[Image Load] ID: s53279434.jpg - Time: 0.00s
[Image Load] ID: s52725543.jpg - Time: 0.00s
[Image Load] ID: s57379366.jpg - Time: 0.00s
[Image Load] ID: s58711995.jpg - Time: 0.00s
[Image Load] ID: s53625776.jpg - Time: 0.00s
[Image Load] ID: s51601762.jpg - Time: 0.00s
[Image Load] ID: s56203761.jpg - Time: 0.00s
[Image Load] ID: s53427313.jpg - Time: 0.00s
[Image Load] ID: s50944285.jpg - Time: 0.00s
[Image Load] ID: s56993819.jpg - Time: 0.00s
[Image Load] ID: s5

In [9]:
# ======================== 5. 모델 평가 ========================
!python test.py \
  --image_dir /content/drive/MyDrive/project/images/test_image \
  --ann_path /content/drive/MyDrive/project/csvs/annotation_50.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 16 \
  --beam_size 3 \
  --topk 32 \
  --num_layers 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/50/test_50 \
  --load /content/drive/MyDrive/project/results/50/train_50/current_checkpoint.pth

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/08/2025 08:51:32 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/project/results/50/train_50/current_checkpoint.pth ...
06/08/2025 08:51:52 - INFO - modules.tester -   Start to evaluate in the test set.
0it [00:00, ?it/s][Image Load] ID: s50010747.jpg - Time: 0.02s
[Image Load] ID: s50111035.jpg - Time: 0.02s
[Image Load] ID: s50014127.jpg - Time:

# 75%

In [2]:
# ==================== 1. 필요한 라이브러리 설치 ====================
!pip install -q nltk scikit-learn

In [3]:
# ==================== 2. GitHub 저장소 클론 ====================
!git clone https://github.com/Vinh-AI/A3Net.git
%cd A3Net

Cloning into 'A3Net'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 121 (delta 5), reused 10 (delta 5), pack-reused 109 (from 2)
Receiving objects: 100% (121/121), 70.26 MiB | 14.88 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/A3Net


In [ ]:
!cp -r /content/drive/MyDrive/project/images/train_image_75 /content/A3Net/data/

cp: cannot access '/content/drive/MyDrive/project/images/train_image_75': Input/output error


In [4]:
# ==================== 3. 경로 설정 (직접 맞게 수정하세요) ====================
image_dir = "/content/A3Net/data/train_image_75"
ann_path = "/content/drive/MyDrive/project/csvs/annotation_75.json"
save_dir = "/content/drive/MyDrive/project/results/75/train_75"

In [ ]:
# ======================== 4. 모델 학습 ========================
!python main.py \
  --image_dir /content/A3Net/data/train_image_75 \
  --ann_path /content/drive/MyDrive/project/csvs/annotation_75.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 64 \
  --epochs 30 \
  --lr_ve 1e-4 \
  --lr_ed 5e-4 \
  --step_size 1 \
  --gamma 0.8 \
  --num_layers 3 \
  --topk 32 \
  --beam_size 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/75/train_75 \
  --log_period 10

python3: can't open file '/content/main.py': [Errno 2] No such file or directory


In [6]:
# ======================== 5. 모델 평가 ========================
!python test.py \
  --image_dir /content/drive/MyDrive/project/images/test_image \
  --ann_path /content/drive/MyDrive/project/csvs/annotation_75.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 16 \
  --beam_size 3 \
  --topk 32 \
  --num_layers 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/75/test_75 \
  --load /content/drive/MyDrive/project/results/75/train_75/current_checkpoint.pth

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/08/2025 08:35:12 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/project/results/75/train_75/current_checkpoint.pth ...
06/08/2025 08:35:14 - INFO - modules.tester -   Start to evaluate in the test set.
0it [00:00, ?it/s][Image Load] ID: s50111035.jpg - Time: 6.59s
[Image Load] ID: s50010747.jpg - Time: 6.59s
[Image Load] ID: s50124332.jpg - Time:

# 100%

In [ ]:
# ==================== 1. 필요한 라이브러리 설치 ====================
!pip install -q nltk scikit-learn

In [ ]:
# ==================== 2. GitHub 저장소 클론 ====================
!git clone https://github.com/Vinh-AI/A3Net.git
%cd A3Net

Cloning into 'A3Net'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 121 (delta 5), reused 10 (delta 5), pack-reused 109 (from 2)
Receiving objects: 100% (121/121), 70.26 MiB | 47.18 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/A3Net


In [ ]:
!cp -r /content/drive/MyDrive/project/images/train_image /content/A3Net/data/

cp: cannot access '/content/drive/MyDrive/project/images/train_image': Input/output error


In [ ]:
# ==================== 3. 경로 설정 (직접 맞게 수정하세요) ====================
image_dir = "/content/A3Net/data/train_image"
ann_path = "/content/drive/MyDrive/project/csvs/annotation.json"
save_dir = "/content/drive/MyDrive/project/results/100/train_100"

In [ ]:
# ======================== 4. 모델 학습 ========================
!python main.py \
  --image_dir /content/A3Net/data/train_image \
  --ann_path /content/drive/MyDrive/project/csvs/annotation.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 64 \
  --epochs 30 \
  --lr_ve 1e-4 \
  --lr_ed 5e-4 \
  --step_size 1 \
  --gamma 0.8 \
  --num_layers 3 \
  --topk 32 \
  --beam_size 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/100/train_100 \
  --log_period 10

python3: can't open file '/content/main.py': [Errno 2] No such file or directory


In [7]:
# ======================== 5. 모델 평가 ========================
!python test.py \
  --image_dir /content/drive/MyDrive/project/images/test_image \
  --ann_path /content/drive/MyDrive/project/csvs/annotation.json \
  --dataset_name mimic_cxr \
  --max_seq_length 100 \
  --threshold 3 \
  --batch_size 16 \
  --beam_size 3 \
  --topk 32 \
  --num_layers 3 \
  --cmm_size 512 \
  --cmm_dim 512 \
  --seed 456789 \
  --save_dir /content/drive/MyDrive/project/results/100/test_100 \
  --load /content/drive/MyDrive/project/results/100/train_100/current_checkpoint.pth

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
06/08/2025 08:43:29 - INFO - modules.tester -   Loading checkpoint: /content/drive/MyDrive/project/results/100/train_100/current_checkpoint.pth ...
06/08/2025 08:43:47 - INFO - modules.tester -   Start to evaluate in the test set.
0it [00:00, ?it/s][Image Load] ID: s50010747.jpg - Time: 0.02s
[Image Load] ID: s50111035.jpg - Time: 0.02s
[Image Load] ID: s50014127.jpg - Tim